# AIPI 590 - XAI | Human-AI Interaction
### A code tutorial explaining LIME in code
### Mariam Khan

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mkhan13/XAI/blob/main/human_ai_interaction.ipynb)

# LIME

LIME (Local Interpretable Model-agnostic Explanations) helps us understand why a machine learning model made a specific prediction by creating a local, interpretable model around one specific instance. From the explainable AI lecture, the LIME Process is:


1. Select instance of interest
2. Perturb your dataset and get black box predictions for perturbed samples
3. Generate a new dataset consisting of perturbed samples (variations of your data) and the corresponding predictions
4. Train an interpretable model, weighted by the proximity of sampled instances to the instance of interest
5. Interpret the local model to explain prediction


In [4]:
# Imports

## Step 1: Select Instance of Interest

## Step 2: Perturb Your Dataset and Get Black-Box Predictions for Perturbed Samples

## Step 3: Generate a New Dataset of Perturbed Samples and Corresponding Predictions

## Step 4: Train an Interpretable Model Weighted by Proximity

## Step 5: Interpret the Local Model to Explain the Prediction